In [1]:
"""
To look at the status of our limb processing
"""

'\nTo look at the status of our limb processing\n'

In [2]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")

import datajoint_utils as du
from importlib import reload
import numpy_utils as nu
import trimesh_utils as tu
import error_detection as ed
import networkx as nx
import networkx_utils as xu
import neuron_utils as nru
import neuron_visualizations as nviz

INFO - 2020-12-10 21:26:06,132 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2020-12-10 21:26:06,133 - settings - Setting database.user to celiib
INFO - 2020-12-10 21:26:06,134 - settings - Setting database.password to newceliipass
INFO - 2020-12-10 21:26:06,138 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2020-12-10 21:26:06,139 - settings - Setting enable_python_native_blobs to True
INFO - 2020-12-10 21:26:06,150 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2020-12-10 21:26:06,366 - settings - Setting enable_python_native_blobs to True


In [3]:
import minfig
import time
import numpy as np
#want to add in a wait for the connection part
random_sleep_sec = np.random.randint(0, 30)
print(f"Sleeping {random_sleep_sec} sec before conneting")
#time.sleep(random_sleep_sec)
print("Done sleeping")

du.config_celii()
du.set_minnie65_config_segmentation(minfig)
du.print_minnie65_config_paths(minfig)

#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

INFO - 2020-12-10 21:26:06,388 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2020-12-10 21:26:06,389 - settings - Setting database.user to celiib
INFO - 2020-12-10 21:26:06,390 - settings - Setting database.password to newceliipass
INFO - 2020-12-10 21:26:06,392 - settings - Setting enable_python_native_blobs to True


Sleeping 23 sec before conneting
Done sleeping
Current path for external_segmentation_path = /mnt/dj-stor01/platinum/minnie65/02
Current path for external_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/meshes
Current path for external_decimated_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/decimated_meshes
Current path for external_skeleton_path = /mnt/dj-stor01/platinum/minnie65/02/skeletons


INFO - 2020-12-10 21:26:06,610 - settings - Setting enable_python_native_blobs to True


In [4]:
len(minnie.LimbStats())

278318

In [5]:
len(minnie.Decomposition.proj() & minnie.LimbStats.proj())

17073

# Downloading the Data

In [6]:
import time
download_start_time = time.time()
total_data_dict = minnie.LimbStats.fetch(as_dict=True)
print(f"Total time for download = {time.time() - download_start_time}")

Total time for download = 26.17853832244873


# Convert to a pandas table

In [ ]:
import pandas as pd
import numpy as np
path_df = pd.DataFrame.from_dict(total_data_dict)

In [ ]:
path_df.to_pickle("neuron_dendrite_path_data")
df_recov = pd.read_pickle("neuron_dendrite_path_data")
df_recov

# Saving the Data

In [ ]:
import system_utils as su
su.decompress_pickle(total_data_dict,"")

# Seeing If Can Get approximately labeled data by the angle (spot checking neurons)

In [8]:
angle_interval = [100,115]
restrict_table = (minnie.LimbStats() & f"soma_angle>={angle_interval[0]} AND soma_angle<={angle_interval[-1]}")
print(f"Length of restricted table = {len(restrict_table)}")
curr_data = restrict_table.fetch("segment_id","limb_idx")

Length of restricted table = 39894


In [9]:
restrict_table

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),limb_idx the limb id path was taken from,path_idx path identifier,soma_angle,n0_width_median_mesh_center,n0_width_no_spine_median_mesh_center,n0_n_spines,n0_total_spine_volume,n0_spine_volume_median,n0_spine_volume_density,n0_skeletal_length,n0_parent_angle,n0_sibling_angle,n1_width_median_mesh_center,n1_width_no_spine_median_mesh_center,n1_n_spines,n1_total_spine_volume,n1_spine_volume_median,n1_spine_volume_density,n1_skeletal_length,n1_parent_angle,n1_sibling_angle
864691134884741114,0,0.25,2,0,101.76,429.53,429.53,1,70511497.64,70511497.64,8447.39,8347.14,46.73,64.97,369.76,351.41,10,779012287.04,71761960.37,52069.05,14961.14,17.98,71.89
864691134884741114,0,0.25,2,1,101.76,429.53,429.53,1,70511497.64,70511497.64,8447.39,8347.14,46.73,64.97,425.5,307.37,109,4804864882.98,30316614.27,42655.63,112643.17,53.93,71.89
864691134884741114,0,0.25,2,2,101.76,560.07,560.07,1,8062477.04,8062477.04,1115.37,7228.49,18.24,64.97,519.87,481.64,4,228441104.21,54306193.97,24131.52,9466.5,22.98,75.12
864691134884741114,0,0.25,2,3,101.76,560.07,560.07,1,8062477.04,8062477.04,1115.37,7228.49,18.24,64.97,408.45,312.89,39,2355101506.47,46894065.26,52496.81,44861.8,58.67,75.12
864691134884744698,0,0.25,1,0,102.37,636.17,553.72,4,310913112.85,55589182.47,29691.23,10471.55,24.78,65.78,362.97,362.97,0,0.0,0.0,0.0,668.25,14.48,49.0
864691134884744698,0,0.25,1,1,102.37,636.17,553.72,4,310913112.85,55589182.47,29691.23,10471.55,24.78,65.78,377.44,283.77,32,2419854415.01,61084704.12,44373.37,54533.94,41.59,49.0
864691134884747002,0,0.25,2,0,111.76,422.39,422.39,0,0.0,0.0,0.0,5835.04,10.41,66.78,373.28,373.28,0,0.0,0.0,0.0,9045.27,19.46,65.7
864691134884747002,0,0.25,2,1,111.76,422.39,422.39,0,0.0,0.0,0.0,5835.04,10.41,66.78,223.88,223.88,2,37447903.75,18723951.87,1465.74,25548.73,48.03,65.7
864691134884747002,0,0.25,2,2,111.76,289.68,277.92,16,713647504.3,35687777.74,11667.45,61165.66,57.21,66.78,60.3,60.3,0,0.0,0.0,0.0,15492.26,49.42,83.16
864691134884747002,0,0.25,3,0,113.16,275.21,275.21,0,0.0,0.0,0.0,4419.1,26.27,93.47,340.48,314.82,5,222074822.17,23457919.88,13244.69,16767.08,30.92,78.5


In [10]:
unique_seg_id_branch_idx = np.unique(np.array(curr_data).T,axis=0)
unique_seg_id_branch_idx

array([[864691134884741114,                  2],
       [864691134884744698,                  1],
       [864691134884747002,                  2],
       ...,
       [864691137196928321,                  3],
       [864691137196928577,                  0],
       [864691137196931137,                  4]])

In [11]:
correct_basal_classification = []
wrong_basal_classification = []

# Looking at an example branch

In [26]:
idx = 4
curr_segment_id,curr_limb_idx = unique_seg_id_branch_idx[idx]
curr_segment_id,curr_limb_idx

(864691134884747002, 4)

In [27]:
import neuron_visualizations as nviz
neuron_obj = (minnie.Decomposition & dict(segment_id=curr_segment_id)).fetch1("decomposition")

Decompressing Neuron in minimal output mode...please wait


In [28]:
nviz = reload(nviz)
print(f"Basal: Segment_id:{curr_segment_id}, Limb: {curr_limb_idx}")
nviz.visualize_neuron(neuron_obj,
                     visualize_type=["mesh"],
                     limb_branch_dict={f"L{curr_limb_idx}":"all"},
                     mesh_resolution="limb",
                     mesh_color="red",
                      mesh_color_alpha=1,
                     mesh_whole_neuron=True,
                     verbose=False)

Basal: Segment_id:864691134884747002, Limb: 4


In [16]:
correct_basal_classification.append([curr_segment_id,curr_limb_idx])

In [ ]:
wrong_basal_classification.append([curr_segment_id,curr_limb_idx])